In [1]:
import pandas as pd

In [2]:
messages = pd.read_csv("./sms-spam-collection/SMSSpamCollection", sep = '\t', names=["label", "message"])
messages.head(5)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [4]:
ps = PorterStemmer()

## Data Cleaning and Preprocessing 

In [5]:
corpus = []
for i in range(len(messages)):
    msg = re.sub("[^a-zA-Z]", " ", messages["message"][i])
    msg = msg.lower()
    msg = msg.split()
    msg = [ps.stem(word) for word in msg if word not in set(stopwords.words("english"))]
    msg = " ".join(msg)
    corpus.append(msg)

## Creating the Bag of words model

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()

In [7]:
y = pd.get_dummies(messages['label'])
y = y.iloc[:,1].values

## train and test split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

## Training model using Naive Bayes Classifier

In [10]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [11]:
y_pred = spam_detect_model.predict(X_test)

In [12]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[946,   9],
       [  8, 152]])

In [14]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9847533632286996